In [ ]:
%matplotlib inline

# Linear regression.

A [LinearRegressor][gemseo.mlearning.regression.algos.linreg.LinearRegressor] is a linear regression model
based on [scikit-learn](https://scikit-learn.org).

!!! info "See also"

    You can find more information about building linear models with scikit-learn on
    [this page](https://scikit-learn.org/stable/modules/linear_model.html).


In [ ]:
from __future__ import annotations

from matplotlib import pyplot as plt
from numpy import array

from gemseo import create_design_space
from gemseo import create_discipline
from gemseo import sample_disciplines
from gemseo.mlearning import create_regression_model

## Problem

In this example,
we represent the function $f(x)=(6x-2)^2\sin(12x-4)$
by the [AnalyticDiscipline][gemseo.disciplines.analytic.AnalyticDiscipline].

!!! quote "References"
      Alexander I. J. Forrester, Andras Sobester, and Andy J. Keane.
      Engineering design via surrogate modelling: a practical guide. Wiley, 2008.



In [ ]:
discipline = create_discipline(
    "AnalyticDiscipline",
    name="f",
    expressions={"y": "(6*x-2)**2*sin(12*x-4)"},
)

and seek to approximate it over the input space



In [ ]:
input_space = create_design_space()
input_space.add_variable("x", lower_bound=0.0, upper_bound=1.0)

To do this,
we create a training dataset with 6 equispaced points:



In [ ]:
training_dataset = sample_disciplines(
    [discipline], input_space, "y", algo_name="PYDOE_FULLFACT", n_samples=6
)

## Basics

### Training

Then,
we train a linear regression model from these samples:



In [ ]:
model = create_regression_model("LinearRegressor", training_dataset)
model.learn()

### Prediction

Once it is built,
we can predict the output value of $f$ at a new input point:



In [ ]:
input_value = {"x": array([0.65])}
output_value = model.predict(input_value)
output_value

as well as its Jacobian value:



In [ ]:
jacobian_value = model.predict_jacobian(input_value)
jacobian_value

### Plotting

Of course,
you can see that the linear model is no good at all here:



In [ ]:
test_dataset = sample_disciplines(
    [discipline], input_space, "y", algo_name="PYDOE_FULLFACT", n_samples=100
)
input_data = test_dataset.get_view(variable_names=model.input_names).to_numpy()
reference_output_data = test_dataset.get_view(variable_names="y").to_numpy().ravel()
predicted_output_data = model.predict(input_data).ravel()
plt.plot(input_data.ravel(), reference_output_data, label="Reference")
plt.plot(input_data.ravel(), predicted_output_data, label="Regression - Basics")
plt.grid()
plt.legend()
plt.show()

## Settings

The [LinearRegressor][gemseo.mlearning.regression.algos.linreg.LinearRegressor] has many options
defined in the [LinearRegressor_Settings][gemseo.mlearning.regression.algos.linreg_settings.LinearRegressor_Settings] Pydantic model.

### Intercept

By default,
the linear model is of the form $a_0+a_1x_1+\ldots+a_dx_d$.
You can set the option `fit_intercept` to `False`
if you want a linear model of the form $a_1x_1+\ldots+a_dx_d$:



In [ ]:
model = create_regression_model(
    "LinearRegressor", training_dataset, fit_intercept=False, transformer={}
)
model.learn()

!!! warning

    This notion applies in the space of transformed variables.
    This is the reason why
    we removed the default transformers by setting `transformer` to `{}`.

We can see the impact of this option in the following visualization:



In [ ]:
predicted_output_data_ = model.predict(input_data).ravel()
plt.plot(input_data.ravel(), reference_output_data, label="Reference")
plt.plot(input_data.ravel(), predicted_output_data, label="Regression - Basics")
plt.plot(input_data.ravel(), predicted_output_data_, label="Regression - No intercept")
plt.grid()
plt.legend()
plt.show()

## Regularization

When the number of samples is small relative to the input dimension,
regularization techniques can save you from overfitting
(a model that is very good at learning but bad at generalization).
The `penalty_level` option is a positive real number
defining the degree of regularization (default: no regularization).
By default,
the regularization technique is the ridge penalty (l2 regularization).
The technique can be replaced by the lasso penalty (l1 regularization)
by setting the `l2_penalty_ratio` option to `0.0`.
When `l2_penalty_ratio` is between 0 and 1,
the regularization technique is the elastic net penalty,
*i.e.* a linear combination of ridge and lasso penalty
parametrized by this `l2_penalty_ratio`.

For example,
we can use the ridge penalty with a level of 1.2



In [ ]:
model = create_regression_model("LinearRegressor", training_dataset, penalty_level=1.2)
model.learn()
predicted_output_data_ = model.predict(input_data).ravel()
plt.plot(input_data.ravel(), reference_output_data, label="Reference")
plt.plot(input_data.ravel(), predicted_output_data, label="Regression - Basics")
plt.plot(input_data.ravel(), predicted_output_data_, label="Regression - Ridge(1.2)")
plt.grid()
plt.legend()
plt.show()

We can see that the coefficient of the linear model is lower due to the penalty.

!!! note

    In the case of a model with many inputs,
    we could have used the lasso penalty
    and seen that some coefficients would have been set to zero.

